In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from skimage.transform import resize
from skimage.io import imread
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras as ks
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import random as rand
import seaborn as sns
import scipy as sci
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import os
import cv2
from math import log10, sqrt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.utils import to_categorical
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
import mtcnn
from PIL import Image
from scipy import ndimage
from tqdm import tqdm
from sklearn.datasets import fetch_lfw_people
from numpy.linalg import norm
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, concatenate, MaxPooling2D, Dropout, UpSampling2D


In [3]:
import os
import numpy as np

In [4]:
datasetpath = "./dataset/"
image_size = 512
patch_size = 64
stride = 32

# Prepare the list to store the ground truth hyperspectral images
groundtruth_images = []

classes = sorted(os.listdir(datasetpath))
print("\n", classes)

dataset_path = datasetpath + classes[0] + "/" + classes[0] + "/"

print("\n", dataset_path)

['balloons_ms', 'beads_ms', 'cd_ms', 'chart_and_stuffed_toy_ms', 'clay_ms', 'cloth_ms', 'egyptian_statue_ms', 'face_ms', 'fake_and_real_beers_ms', 'fake_and_real_food_ms', 'fake_and_real_lemon_slices_ms', 'fake_and_real_lemons_ms', 'fake_and_real_peppers_ms', 'fake_and_real_strawberries_ms', 'fake_and_real_sushi_ms', 'fake_and_real_tomatoes_ms', 'feathers_ms', 'flowers_ms', 'glass_tiles_ms', 'hairs_ms', 'jelly_beans_ms', 'oil_painting_ms', 'paints_ms', 'photo_and_face_ms', 'pompoms_ms', 'real_and_fake_apples_ms', 'real_and_fake_peppers_ms', 'sponges_ms', 'stuffed_toys_ms', 'superballs_ms', 'thread_spools_ms', 'watercolors_ms']

 ./dataset/balloons_ms/balloons_ms/


In [5]:
for i in range(0, 31):
    print(i)
    dataset_path = datasetpath + classes[i] + "/" + classes[i] + "/"
    print(dataset_path)
    
   # Iterate over the images in the dataset directory
    for filename in sorted(os.listdir(dataset_path)):
        if filename.endswith(".png"):
            image_path = os.path.join(dataset_path, filename)
            image = imread(image_path)
            # Crop the hyperspectral image into smaller patches
            for i in range(0, image_size - patch_size + 1, stride):
                for j in range(0, image_size - patch_size + 1, stride):
                    patch = image[i : i + patch_size, j : j + patch_size]
                    groundtruth_images.append(patch)
    
    # Convert the list of ground truth patches to a numpy array
    groundtruth_arrays = np.array(groundtruth_images)

    # Reshape the ground truth images to match the specified dimensions
    groundtruth_arrays = groundtruth_arrays.reshape(-1, patch_size, patch_size, 31)

    # Print the shape of the ground truth hyperspectral images array
    print("Ground Truth Hyperspectral Images Shape:", groundtruth_arrays.shape)


0
./dataset/balloons_ms/balloons_ms/
Ground Truth Hyperspectral Images Shape: (225, 64, 64, 31)
1
./dataset/beads_ms/beads_ms/
Ground Truth Hyperspectral Images Shape: (450, 64, 64, 31)
2
./dataset/cd_ms/cd_ms/
Ground Truth Hyperspectral Images Shape: (675, 64, 64, 31)
3
./dataset/chart_and_stuffed_toy_ms/chart_and_stuffed_toy_ms/
Ground Truth Hyperspectral Images Shape: (900, 64, 64, 31)
4
./dataset/clay_ms/clay_ms/
Ground Truth Hyperspectral Images Shape: (1125, 64, 64, 31)
5
./dataset/cloth_ms/cloth_ms/
Ground Truth Hyperspectral Images Shape: (1350, 64, 64, 31)
6
./dataset/egyptian_statue_ms/egyptian_statue_ms/
Ground Truth Hyperspectral Images Shape: (1575, 64, 64, 31)
7
./dataset/face_ms/face_ms/
Ground Truth Hyperspectral Images Shape: (1800, 64, 64, 31)
8
./dataset/fake_and_real_beers_ms/fake_and_real_beers_ms/
Ground Truth Hyperspectral Images Shape: (2025, 64, 64, 31)
9
./dataset/fake_and_real_food_ms/fake_and_real_food_ms/
Ground Truth Hyperspectral Images Shape: (2250, 64, 

In [ ]:
from scipy.ndimage import uniform_filter
# Define the size of the averaging filter
filter_size = (8, 8)

# Apply the averaging filter on each band of the ground truth hyperspectral images
lowres_hsi_images = np.zeros((groundtruth_arrays.shape[0], 8, 8, 31))
for i in range(groundtruth_arrays.shape[0]):
    for j in range(groundtruth_arrays.shape[3]):
        band = groundtruth_arrays[i, :, :, j]
        lowres_band = uniform_filter(band, size=filter_size)
        lowres_hsi_images[i, :, :, j] = lowres_band[::8, ::8]

In [ ]:
# Print the shape of the low-resolution HSI images array
print("Low-Resolution HSI Images Shape:", lowres_hsi_images.shape)

Low-Resolution HSI Images Shape: (6300, 8, 8, 31)


In [ ]:
# Apply the 1x1 filters to generate the high-resolution RGB images
highres_rgb_images = np.zeros((groundtruth_arrays.shape[0], 64, 64, 3))
for i in range(groundtruth_arrays.shape[0]):
    for j in range(3):
        start_band = j * 10
        end_band = (j + 1) * 10
        rgb_band_avg = np.mean(groundtruth_arrays[i, :, :, start_band:end_band], axis=-1)
        highres_rgb_images[i, :, :, j] = rgb_band_avg

In [ ]:
# Print the shape of the high-resolution RGB images array
print("High-Resolution RGB Images Shape:", highres_rgb_images.shape)

High-Resolution RGB Images Shape: (6300, 64, 64, 3)


In [ ]:
# Split the dataset into training and testing sets
train_ratio = 0.8
num_samples = groundtruth_arrays.shape[0]
num_train_samples = int(train_ratio * num_samples)
y_train = groundtruth_arrays[:num_train_samples]
train_lowres_hsi = lowres_hsi_images[:num_train_samples]
train_highres_rgb = highres_rgb_images[:num_train_samples]
test_lowres_hsi = lowres_hsi_images[num_train_samples:]
test_highres_rgb = highres_rgb_images[num_train_samples:]
y_test = groundtruth_arrays[num_train_samples:]

In [ ]:
# Normalize the data (optional but recommended)
train_lowres_hsi = train_lowres_hsi / 255.0
train_highres_rgb = train_highres_rgb / 255.0
test_lowres_hsi = test_lowres_hsi / 255.0
test_highres_rgb = test_highres_rgb / 255.0

In [ ]:

# Define the input shapes
msi_input_shape = (8, 8, 31)
hsi_input_shape = (64, 64, 3)

# Define the MSI branch
msi_input = Input(shape=msi_input_shape)
msi_conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(msi_input)
msi_conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(msi_conv1)
msi_pooling = MaxPooling2D((8, 8))(msi_conv2)  # Adjust the pooling size
msi_dropout = Dropout(0.2)(msi_pooling)

# Define the HSI branch
hsi_input = Input(shape=hsi_input_shape)
hsi_conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(hsi_input)
hsi_conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(hsi_conv1)
hsi_pooling = MaxPooling2D((2, 2))(hsi_conv2)
hsi_dropout = Dropout(0.2)(hsi_pooling)

# Upsample MSI features to match HSI features
msi_upsampled = UpSampling2D((32, 32))(msi_dropout)  # Adjust the upsampling size

# Concatenate the outputs of both branches
fusion = concatenate([msi_upsampled, hsi_dropout], axis=-1)

# Add more layers for fusion
fusion_conv1 = Conv2D(128, (3, 3), activation='relu', padding='same')(fusion)
fusion_conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(fusion_conv1)
fusion_upsampling = UpSampling2D((2, 2))(fusion_conv2)
fusion_output = Conv2D(31, (3, 3), activation='relu', padding='same')(fusion_upsampling)

# Create the model
model = Model(inputs=[msi_input, hsi_input], outputs=fusion_output)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8, 8, 31)]   0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 8, 8, 64)     17920       ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d_1 (Conv2D)              (None, 8, 8, 64)     36928       ['conv2d[0][0]']                 
                                                                                              

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the necessary parameters
batch_size = 32
epochs = 10
validation_split = 0.2

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model
history = model.fit(
    x=[train_lowres_hsi, train_highres_rgb],
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
    callbacks=[early_stopping]
)

Epoch 1/10
126/126 [==============================] - 241s 2s/step - loss: 42872932.0000 - mae: 3611.0830 - val_loss: 50494928.0000 - val_mae: 4208.1172
Epoch 2/10
126/126 [==============================] - 237s 2s/step - loss: 31931624.0000 - mae: 2976.8145 - val_loss: 50120464.0000 - val_mae: 4211.1074
Epoch 3/10
126/126 [==============================] - 1261s 10s/step - loss: 31414804.0000 - mae: 2941.9419 - val_loss: 49758796.0000 - val_mae: 4165.8525
Epoch 4/10
116/126 [==========================>...] - ETA: 15s - loss: 29016504.0000 - mae: 2827.7131

In [ ]:
#compile the model with the optimizer and loss function
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Evaluate the model on the test set
model.evaluate(x=[test_lowres_hsi, test_highres_rgb], y=y_test)

# Predict on the test set
y_pred = model.predict([test_lowres_hsi, test_highres_rgb])

# Print the shape of the predicted array
print("Predicted Array Shape:", y_pred.shape)

# Plot the first sample in the test set
plt.figure(figsize=(10, 10))
plt.imshow(y_pred[0])
plt.show()
 

41/41 [==============================] - 8s 184ms/step


ResourceExhaustedError: {{function_node __wrapped__ConcatV2_N_41_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[1305,64,64,31] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:ConcatV2] name: concat

In [ ]:
#plot the results
#debug this code above there are errors

import matplotlib.pyplot as plt
import numpy as np



# Get the model predictions
predictions = model.predict([test_lowres_hsi, test_highres_rgb])  # Replace with your MSI and HSI data

# Plot a random sample of the ground truth RGB images and their predictions
fig, axes = plt.subplots(nrows=2, ncols=5, figsize=(15, 6))
for i, ax in enumerate(axes.flat):
    idx = np.random.randint(0, len(predictions))
    ax.imshow(predictions[idx])
    ax.set_title(f'Image #{idx}')
    ax.axis('off')
plt.tight_layout()
plt.show()








 7/41 [====>.........................] - ETA: 6s

: 

: 